In [1]:
from kan.hypothesis import *
from torchvision import transforms
import torch
import torch.nn.functional as F
from kan import *
import h5py
import copy
import sympy as sp
dtype = torch.get_default_dtype()
from sympy import latex
import pickle

In [2]:
data_dir = "./"
filename = "sg2-data.h5"
checkpoint_dir = "./saved_models/"
log_dir = './logs/'
band_idx = 0
gidxs = [1, 2, 3, 4]
only_obstructed = False
only_topological = False
binary_classification = False#True
no_penalize_last = True
bias = False
numrs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
file = h5py.File(data_dir + filename, 'r')
print("file keys: ", file['sg2/1/'].keys());

file keys:  <KeysViewHDF5 ['epsilon_Gs-gidx=1', 'epsilon_Gs-gidx=2', 'epsilon_Gs-gidx=3', 'epsilon_Gs-gidx=4', 'frequencies-gidx=1-mode=tm', 'frequencies-gidx=2-mode=tm', 'frequencies-gidx=3-mode=tm', 'frequencies-gidx=4-mode=tm', 'symmetry-gidx=1-mode=tm', 'symmetry-gidx=2-mode=tm', 'symmetry-gidx=3-mode=tm', 'symmetry-gidx=4-mode=tm']>


In [4]:
nGs = 10
gidxs = [1, 2, 3, 4]
wps = ['1a', '1b', '1c', '1d']
wps2 = ['1a', '1c', '1b', '1d'] # switch 1b and 1c due to convention of how Gvectors are stored

symmetry_data = torch.zeros(10000 * len(gidxs) * len(wps))
input_data = torch.zeros(10000 * len(gidxs) * len(wps), nGs)

In [5]:
for (widx, (wp1, wp2)) in enumerate(zip(wps, wps2)):
    print(f"Wyckoff index: {widx}")
    sym_vec_phases = file[f'sg2/symmetry_vector_phases/{wp1}'][()]
    epsilon_G_phases = file[f'sg2/epsilon_G_phases/{wp2}'][()].real
    for gidx in gidxs:
        print(f"gidx: {gidx}")
        for id in range(1, 10001):
            real_id = (id-1) + (gidx-1)*10000 + len(gidxs)*10000*widx
            if (id % 1000 == 0): 
                print(id)
            symmetry_before_aug = file[f'sg2/{id}/symmetry-gidx={gidx}-mode=tm'][()][band_idx]
            if band_idx == 0:
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] - 8
            else: 
                symmetry_data[real_id] =  sym_vec_phases[symmetry_before_aug] 

            fourier_data_before_aug = file[f'sg2/{id}/epsilon_Gs-gidx={gidx}'][()][0:nGs].real
            fourier_data = fourier_data_before_aug * epsilon_G_phases
            input_data[real_id, :] = torch.tensor([*fourier_data])

Wyckoff index: 0
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 1
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 2
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Wyckoff index: 3
gidx: 1
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 2
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 3
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
gidx: 4
1000
2000
3000
4000
5000
6000
7000
8000

In [6]:
shuffled_indices = torch.randperm(10000 * len(gidxs) * len(wps))
input_data_shuffled = input_data[shuffled_indices, :]
symmetry_data_shuffled = symmetry_data[shuffled_indices]

In [21]:
symmetry_data.long()

tensor([7, 5, 7,  ..., 0, 0, 4])

In [7]:
dataset_kan = {}
n_train = 80000#int(input_data_shuffled.shape[0]*2/3)
nGs = 10
nGstart = 1
dataset_kan['train_input'] = input_data_shuffled[0:n_train, [*range(nGstart, nGs)]].to(device)
dataset_kan['test_input'] = input_data_shuffled[n_train:, [*range(nGstart, nGs)]].to(device)
dataset_kan['train_label'] = symmetry_data_shuffled[0:n_train].long().to(device)
dataset_kan['test_label'] = symmetry_data_shuffled[n_train:].long().to(device)

In [13]:
model = KAN.loadckpt('./band1');

In [14]:
print((model(dataset_kan['train_input']).argmax(1) == dataset_kan['train_label']).sum()/80000)
print((model(dataset_kan['test_input']).argmax(1) == dataset_kan['test_label']).sum()/80000)

tensor(0.9970, device='cuda:0')
tensor(0.9971, device='cuda:0')


In [51]:
model = model.prune_input(threshold=0.25)

keep: [True, True, True, False, False, False, False, False, False]
saving model version 1.2


In [52]:
model = model.prune_node(threshold=0.2)

saving model version 1.3


In [53]:
model = model.refine(1)

saving model version 1.4


In [54]:
model.width_in

[3, 4, 8]

In [61]:
n_class = 16
if band_idx == 0:
    n_class = 8
lr = 1
def train_acc():
    return torch.mean((torch.argmax(model(dataset_kan['train_input']), dim=1) == dataset_kan['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model(dataset_kan['test_input']), dim=1) == dataset_kan['test_label']).type(dtype))
    
results = model.fit(dataset_kan, opt="LBFGS", steps=100, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(), lr=lr);
model.attribute()

| train_loss: 3.36e-01 | test_loss: 3.40e-01 | reg: 0.00e+00 | : 100%|█| 100/100 [01:12<00:00,  1.37

saving model version 1.7


In [62]:
print("Train accuracy: ", round(results['train_acc'][-1]*100, 2),  " Test accuracy: ", round(results['test_acc'][-1]*100, 2))

Train accuracy:  98.63  Test accuracy:  98.66


In [58]:
lib = ['x','x^2','x^3','x^4','exp','tanh','sin', 'cos', 'gaussian']

In [59]:
model.auto_symbolic(a_range=(-10, 10), b_range=(-10, 10), weight_simple=0.1, lib=lib)

fixing (0,0,0) with x^2, r2=0.9999682903289795, c=2
fixing (0,0,1) with sin, r2=1.0000003576278687, c=2
fixing (0,0,2) with sin, r2=0.9999534487724304, c=2
fixing (0,0,3) with x^2, r2=1.000000238418579, c=2
fixing (0,1,0) with x, r2=0.9999997019767761, c=1
fixing (0,1,1) with gaussian, r2=0.9999930262565613, c=3
fixing (0,1,2) with x^2, r2=1.000000238418579, c=2
fixing (0,1,3) with x^2, r2=0.9999995827674866, c=2
fixing (0,2,0) with x^2, r2=1.000000238418579, c=2
fixing (0,2,1) with tanh, r2=1.0000001192092896, c=3
fixing (0,2,2) with sin, r2=1.0000003576278687, c=2
fixing (0,2,3) with gaussian, r2=0.9999961256980896, c=3
fixing (1,0,0) with cos, r2=0.973871648311615, c=2
fixing (1,0,1) with cos, r2=0.974907636642456, c=2
fixing (1,0,2) with cos, r2=0.9743461012840271, c=2
fixing (1,0,3) with cos, r2=0.9739751815795898, c=2
fixing (1,0,4) with cos, r2=0.9742485880851746, c=2
fixing (1,0,5) with cos, r2=0.9739978909492493, c=2
fixing (1,0,6) with cos, r2=0.9739285111427307, c=2
fixing (

In [63]:
print((model(dataset_kan['train_input']).argmax(1) == dataset_kan['train_label']).sum()/80000)
print((model(dataset_kan['test_input']).argmax(1) == dataset_kan['test_label']).sum()/80000)

tensor(0.9863, device='cuda:0')
tensor(0.9866, device='cuda:0')


In [64]:
formulas = model.symbolic_formula()[0]

In [65]:
formulas[0]

-129.609390258789*sin(37.8580876521896*(-0.00915208486399082*x_1 - 1)**2 + 0.269286522207325*(-0.140214744447865*x_2 - 1)**2 - 45.5379867430811 + 0.0295338481017029*exp(-1.59143428518297*(1 - 0.378845582505749*x_3)**2)) + 2969.80053710938*cos(0.773116518248274*x_2 - 0.0398802686707641*(1 - 0.0490486633551681*x_3)**2 + 0.00532111946294038*(-0.95345625436313*x_1 - 1)**2 - 9.71947414449351) - 2658.01196289063*cos(0.0106595408547071*(1 - 0.0346471348982541*x_2)**2 + 0.0011032851606467*sin(0.909361004829407*x_1 + 7.79623985290527) - 11.0233404574251*sin(0.0548879019916058*x_3 + 9.37358379364014) + 10.2766956943483) - 2359.97509765625*cos(8.39792568003645*sin(0.0267096552997828*x_1 + 9.29491710662842) + 0.00455429082624748*tanh(0.481528669595718*x_3 - 0.681991636753082) - 10.8454652682932 + 0.00257975759753393*exp(-1.60802687653182*(1 - 0.392716174946834*x_2)**2)) - 2084.42408752441

In [66]:
idx = 0
N = 100
symbolic_predictions = torch.zeros(N) 
for idx in range(0, N):
    x, y, z = dataset_kan['train_input'][idx, 0:3]
    formula_outputs = np.zeros(8)
    for formula_idx in range(0, 8):
        formula_output = formulas[formula_idx].subs('x_1', x).subs('x_2', y).subs('x_3', z)
        formula_outputs[formula_idx] = formula_output
    symbolic_predictions[idx] = formula_outputs.argmax()   

In [70]:
(symbolic_predictions == dataset_kan['train_label'].cpu()[0:100]).sum()

tensor(99)

In [81]:
for topological_class in range(0, 8):
    with open(f"formula-class{topological_class}.pkl", "wb") as f:
        pickle.dump(formulas[topological_class], f)

In [8]:
all_formulas = []
for topological_class in range(0, 8): 
    with open(f"formula-class{topological_class}.pkl", "rb") as f:
        loaded_formula = pickle.load(f)
        all_formulas.append(loaded_formula)

In [9]:
def formula_eval(t, formula_idx):
    x, y, z = t
    return float(all_formulas[formula_idx].subs('x_1', x).subs('x_2', y).subs('x_3', z))
def formula_loss(t, loss_vector):
    #loss_vector = np.array([7, -1, -1, -1, -1, -1, -1, -1])
    formula_results = [formula_eval(t, n) for n in range(0, 8)]
    return (formula_results * loss_vector).sum()

In [15]:
N = 1000
formula_predictions = torch.zeros((N, 8), dtype=float)
for i in range(0, N): 
    for j in range(0, 8):
        y = formula_eval(dataset_kan['train_input'][i:i+1, 0:3][0], j)
        formula_predictions[i, j] = y

In [18]:
x = (formula_predictions.argmax(1) == dataset_kan['train_label'].cpu()[0:N]).sum()/N

In [19]:
x.item()

0.9919999837875366